We start defining the octahedron as a `Sagemath` object.

In [1]:
P = polytopes.octahedron()
P.plot(viewer='threejs', frame=False)

Graphics3d Object

We consider the ordered lists of vertices and triangles.

In [4]:
vertices = list(P.vertices())
vertices.sort()
triangles = [list(t.vertices()) for t in P.faces(2)]
for tr in triangles:
    tr.sort()
triangles.sort()
print("There are", len(vertices), "vertices")
print("There are", len(triangles), "triangles")

There are 6 vertices
There are 8 triangles


We consider the list `B` of *ordered* pairs of triangles intersecting at one vertex and such that the other vertices are opposed. For each vertex we have two such pairs, i.e., we have $12$ pairs.

In [5]:
B = []
for tr in triangles:
    for v in tr:
        tr1 = [v]
        op = [-w.vector() for w in tr if w != v]
        tr1 += [Polyhedron([w]).vertices()[0] for w in op]
        tr1.sort()
        par = sorted([tr, tr1])
        if par not in B:
            B.append(par)
len(B)

12

We construct a dictionary `dic:` to each vertex it associates the *ordered* list of pairs of triangles in `B` intersecting at this vertex.

In [6]:
dic = {v: [[t1, t2] for t1, t2 in B if v in t1 and v in t2] for v in vertices}
[len(dic[v]) for v in vertices]

[2, 2, 2, 2, 2, 2]

Recall that each smoothing is associated to a choice for each vertex of a couple of opposed triangles at this vertex, for which the former triangles acquire a new edge from this vertex. This is coded in a list `C` corresponding to the $2^6=64$ possible smoothings.

In [7]:
C = [[[v] +[dic[v][w[j]]] for j, v in enumerate(vertices)] for w in GF(2)^6]
C.sort()
len(C)

64

We consider the automorphism group of the octahedron, represented as a permutation group of the vertices.

In [10]:
G0 = P.combinatorial_automorphism_group(vertex_graph_only=True)
G0.order()

48

The group $G_0$ acts on `C;` we construct the orbits of this action.

In [11]:
D = [c for c in C]
orbits = []
while D:
    c0 = D[0]
    orb = []
    for g0 in G0:
        c1 = sorted([[g0(v), sorted([sorted([g0(c) for c in b]) for b in a])] for v, a in c0])
        if c1 not in orb:
            orb.append(c1)
    orb.sort()
    orbits.append(orb)
    D = [c for c in D if c not in orb]

In [10]:
[len(orb) for orb in orbits]

[4, 24, 12, 2, 12, 6, 4]

For further use we want to define a dictionary such that to a pair $(v,t_1)$, where $v$ is a vertex and $t_1$ is a triangle adjacent to the vertex, it returns $(v,t_2)$ where $t_2$ is the triangle adjacent to $v$ such that the other vertices of $t_2$ are opposed to the other ones in $t_1$.

In [12]:
cambio = dict()
for v in vertices:
    t1, t2 = dic[v]
    cambio[v, tuple(tuple(c) for c in t1)] = (v, t2)
    cambio[v, tuple(tuple(c) for c in t2)] = (v, t1)

In order to print the information, we assign indices to vertices and triangles.

In [13]:
for j, v in enumerate(vertices):
    print(v, " with index ", j)

A vertex at (-1, 0, 0)  with index  0
A vertex at (0, -1, 0)  with index  1
A vertex at (0, 0, -1)  with index  2
A vertex at (0, 0, 1)  with index  3
A vertex at (0, 1, 0)  with index  4
A vertex at (1, 0, 0)  with index  5


In [14]:
for j, tr in enumerate(triangles):
    tr0 = [vertices.index(v) for v in tr]
    print(tr0, " with index ", j)

[0, 1, 2]  with index  0
[0, 1, 3]  with index  1
[0, 2, 4]  with index  2
[0, 3, 4]  with index  3
[1, 2, 5]  with index  4
[1, 3, 5]  with index  5
[2, 4, 5]  with index  6
[3, 4, 5]  with index  7


We give the *index* version of some objects.

In [15]:
C_index = [[[vertices.index(v)] + [[triangles.index(c) for c in b]] for v, b in a] for a in C]
orbitas_index = [[[[vertices.index(v)] +[[triangles.index(e) for e in t]] for v, t in a] for a in orb] for orb in orbits]
dic_index = {vertices.index(v): [[triangles.index(a) for a in e] for e in dic[v]] for v in vertices}
cambio_index = dict()
for j in range(6):
    a, b = dic_index[j]
    cambio_index[j, tuple(a)] = [j, b]
    cambio_index[j, tuple(b)] = [j, a]

Let us see representatives of each orbit.

In [16]:
for orb in orbitas_index:
    print(orb[0])

[[0, [0, 3]], [1, [0, 5]], [2, [0, 6]], [3, [1, 7]], [4, [2, 7]], [5, [4, 7]]]
[[0, [0, 3]], [1, [0, 5]], [2, [0, 6]], [3, [1, 7]], [4, [2, 7]], [5, [5, 6]]]
[[0, [0, 3]], [1, [0, 5]], [2, [0, 6]], [3, [1, 7]], [4, [3, 6]], [5, [5, 6]]]
[[0, [0, 3]], [1, [0, 5]], [2, [0, 6]], [3, [3, 5]], [4, [3, 6]], [5, [5, 6]]]
[[0, [0, 3]], [1, [0, 5]], [2, [2, 4]], [3, [1, 7]], [4, [2, 7]], [5, [5, 6]]]
[[0, [0, 3]], [1, [0, 5]], [2, [2, 4]], [3, [1, 7]], [4, [3, 6]], [5, [5, 6]]]
[[0, [0, 3]], [1, [0, 5]], [2, [2, 4]], [3, [3, 5]], [4, [2, 7]], [5, [4, 7]]]


Given a smoothing $\sigma$, we can construct six smoothings $\sigma_1,\dots,\sigma_6$, such that they are obtained from $\sigma$ by flipping exactly the smoothing at one vertex. We list in which orbits are each $\sigma_i$ where $\sigma$ is a representative of each orbit.

In [17]:
flips = []
for orb in orbitas_index:
    flip = []
    caso0 = orb[0]
    for j, lst in caso0:
        aux = [a for a in caso0]
        aux[j] = cambio_index[j, tuple(lst)]
        esta = False
        k = 0
        while not esta:
            esta = aux in orbitas_index[k]
            if not esta:
                k += 1
        flip.append(k)
    flips.append(flip)
for j, lst in enumerate(flips):
    print("The flips of a smoothing in the orbit", j, "are in the orbits", lst)

The flips of a smoothing in the orbit 0 are in the orbits [1, 1, 1, 1, 1, 1]
The flips of a smoothing in the orbit 1 are in the orbits [6, 4, 4, 2, 2, 0]
The flips of a smoothing in the orbit 2 are in the orbits [1, 1, 5, 3, 1, 1]
The flips of a smoothing in the orbit 3 are in the orbits [2, 2, 2, 2, 2, 2]
The flips of a smoothing in the orbit 4 are in the orbits [1, 5, 1, 1, 5, 1]
The flips of a smoothing in the orbit 5 are in the orbits [4, 4, 2, 2, 4, 4]
The flips of a smoothing in the orbit 6 are in the orbits [1, 1, 1, 1, 1, 1]


In [269]:
[len(orb) for orb in orbitas_index]

[4, 24, 12, 2, 12, 6, 4]

Each smoothing has a dual one (flip at all the vertices). The first and last orbits are dual, while the other ones are self-dual.

In [26]:
dual = []
for orb in orbitas_index:
    caso0 = orb[0]
    caso1 = [[j, sorted(cambio_index[j, tuple(lst)][1])] for j, lst in caso0]
    esta = False
    k = 0
    while not esta:
        esta = caso1 in orbitas_index[k]
        if not esta:
            k += 1
    dual.append(k)
dual

[6, 1, 2, 3, 4, 5, 0]

For each orbit we count the number of triangles, quadrangles, etc.

In [29]:
polygons = []
for orb in orbitas_index:
    caso = orb[0]
    dic = {l:0 for l in range(3, 8)}
    for k in range(8):
        A = [j for j in range(6) if k in caso[j][1]]
        dic[len(A) + 3] += 1
    dic = {l: dic[l] for l in dic if dic[l]}
    polygons.append(dic)
polygons

[{4: 6, 6: 2},
 {3: 1, 4: 3, 5: 3, 6: 1},
 {3: 2, 4: 2, 5: 2, 6: 2},
 {3: 4, 6: 4},
 {4: 4, 5: 4},
 {4: 4, 5: 4},
 {3: 2, 5: 6}]